## Partie 1


## Marché en temps réels

In [ ]:
# Importation des bibliothèques nécessaires
import streamlit as st
import pandas as pd
import time
import random  # Pour générer des données aléatoires
!pip install yfinance streamlit pandas
import yfinance as yf
import pandas as pd
import time


In [14]:
def fetch_market_data_fixed(indices):
    data = []
    for nom, symbole in indices.items():
        try:
            # Récupération de l'historique sur 5 jours
            ticker = yf.Ticker(symbole)
            historique = ticker.history(period="5d")

            # Vérification que nous avons au moins 2 jours de données
            if len(historique) >= 2:
                # Calcul du prix actuel et de la variation
                prix = historique["Close"][-1]  # Dernier prix de clôture
                prix_precedent = historique["Close"][-2]  # Prix de clôture précédent
                variation = ((prix - prix_precedent) / prix_precedent) * 100
                variation_str = f"{variation:.2f}%"
            else:
                prix = "N/A"
                variation_str = "N/A"

            # Ajout des données pour cet indice
            data.append([nom, round(prix, 2) if isinstance(prix, (float, int)) else prix, variation_str])
        except Exception as e:
            print(f"Erreur pour {nom} ({symbole}) : {str(e)}")
            data.append([nom, "N/A", "N/A"])
    
    # Création d'un DataFrame
    return pd.DataFrame(data, columns=["Indice", "Prix Actuel", "Variation (%)"])

# Liste des indices
indices = {
    "Dow Jones": "^DJI",
    "S&P 500": "^GSPC",
    "NASDAQ": "^IXIC",
    "CAC 40": "^FCHI",
    "DAX": "^GDAXI"
}

# Récupération des données
df = fetch_market_data_fixed(indices)
df

/var/folders/wk/bl83_6q12zxbwn9trgkm74gw0000gn/T/ipykernel_72833/4152157359.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prix = historique["Close"][-1]  # Dernier prix de clôture
/var/folders/wk/bl83_6q12zxbwn9trgkm74gw0000gn/T/ipykernel_72833/4152157359.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prix_precedent = historique["Close"][-2]  # Prix de clôture précédent
/var/folders/wk/bl83_6q12zxbwn9trgkm74gw0000gn/T/ipykernel_72833/4152157359.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent 

,Indice,Prix Actuel,Variation (%)
0,Dow Jones,44918.15,0.44%
1,S&P 500,6029.42,0.51%
2,NASDAQ,19187.41,0.67%
3,CAC 40,7198.71,0.27%
4,DAX,19548.42,0.63%


In [15]:
import time

# Fonction pour rafraîchir les données
def simulate_refresh():
    print("Données rafraîchies à :", time.strftime('%H:%M:%S'))
    time.sleep(2)  # Pause simulant un intervalle

# Simuler un rafraîchissement
for i in range(3):  # Rafraîchit 3 fois
    simulate_refresh()

Données rafraîchies à : 16:59:21
Données rafraîchies à : 16:59:23
Données rafraîchies à : 16:59:25


In [22]:
# Fonction pour styliser les variations
def highlight_variation(val):
    try:
        val_float = float(val.strip('%'))
        if val_float > 0:
            color = 'green'
            icon = '🔼'
        elif val_float < 0:
            color = 'red'
            icon = '🔽'
        else:
            color = 'grey'
            icon = ''
        return f"{icon} {val} ({color})"
    except:
        return val

# Appliquer la stylisation
df["Variation Stylisée"] = df["Variation (%)"].apply(highlight_variation)
df

,Indice,Prix Actuel,Variation (%),Variation Stylisée
0,Dow Jones,44918.15,0.44%,🔼 0.44% (green)
1,S&P 500,6029.42,0.51%,🔼 0.51% (green)
2,NASDAQ,19187.41,0.67%,🔼 0.67% (green)
3,CAC 40,7198.71,0.27%,🔼 0.27% (green)
4,DAX,19548.42,0.63%,🔼 0.63% (green)


In [23]:
# Statistiques sur les variations récupérées
stats = df[df["Variation (%)"] != "N/A"].copy()
stats["Variation Numérique"] = stats["Variation (%)"].apply(lambda x: float(x.strip('%')))
summary = stats["Variation Numérique"].describe()

# Affichage
print("Statistiques sur les variations récupérées :")
print(summary)

Statistiques sur les variations récupérées :
count    5.000000
mean     0.504000
std      0.159937
min      0.270000
25%      0.440000
50%      0.510000
75%      0.630000
max      0.670000
Name: Variation Numérique, dtype: float64


## Recherche actions/Comparaisons d'action 

In [26]:
# Importation des bibliothèques nécessaires
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go

# Description de l'objectif
"""
Comment récupérer les données financières pour une action spécifique via Yahoo Finance (yfinance),
les analyser et les visualiser avec Plotly.
"""

'\nComment récupérer les données financières pour une action spécifique via Yahoo Finance (yfinance),\nles analyser et les visualiser avec Plotly.\n'

In [27]:
# Paramètres utilisateur simulés
symbole = "AAPL"  # Exemple : symbole pour Apple
periode = "1y"  # Période d'historique (par exemple, 1 an)

# Fonction pour récupérer l'historique des prix d'une action
def fetch_stock_data(symbole, periode):
    try:
        # Récupérer les données historiques
        ticker = yf.Ticker(symbole)
        historique = ticker.history(period=periode)
        
        # Vérifier si les données sont disponibles
        if historique.empty:
            print(f"Aucune donnée trouvée pour le symbole {symbole}.")
            return None, None
        
        # Récupérer les informations générales
        info = ticker.info
        return historique, info
    except Exception as e:
        print(f"Erreur lors de la récupération des données : {str(e)}")
        return None, None

# Appel de la fonction
historique, info = fetch_stock_data(symbole, periode)
historique.head()  # Aperçu des données

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-11-29 00:00:00-05:00,189.969332,191.153533,188.048749,188.446793,43014200,0.0,0.0
2023-11-30 00:00:00-05:00,188.914492,189.392163,187.272542,189.023956,48794400,0.0,0.0
2023-12-01 00:00:00-05:00,189.402111,190.626110,188.307468,190.307678,45679300,0.0,0.0
2023-12-04 00:00:00-05:00,189.053822,189.123488,186.536157,188.506500,43389500,0.0,0.0
2023-12-05 00:00:00-05:00,189.282693,193.452254,189.252826,192.477036,66628400,0.0,0.0


In [28]:
if info:
    print(f"Nom complet : {info.get('longName', 'N/A')}")
    print(f"Secteur : {info.get('sector', 'N/A')}")
    print(f"Industrie : {info.get('industry', 'N/A')}")
    print(f"Site Web : {info.get('website', 'N/A')}")

Nom complet : Apple Inc.
Secteur : Technology
Industrie : Consumer Electronics
Site Web : https://www.apple.com


In [29]:
if info:
    print(f"Prix actuel : ${info.get('currentPrice', 'N/A')}")
    print(f"Variation sur 1 an : {info.get('52WeekChange', 0):.2%}")
    print(f"Capitalisation boursière : ${info.get('marketCap', 'N/A'):,}")
    print(f"PER (TTM) : {info.get('trailingPE', 'N/A')}")
    print(f"Bénéfice par action (EPS) : ${info.get('trailingEps', 'N/A')}")
    print(f"Dividende : {info.get('dividendYield', 0):.2%}")

Prix actuel : $236.131
Variation sur 1 an : 22.85%
Capitalisation boursière : $3,569,308,925,952
PER (TTM) : 38.837334
Bénéfice par action (EPS) : $6.08
Dividende : 0.43%


In [30]:
if historique is not None:
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=historique.index,
        y=historique["Close"],
        mode='lines',
        name='Prix de clôture'
    ))
    fig.update_layout(
        title=f"Historique du prix de l'action {symbole.upper()}",
        xaxis_title="Date",
        yaxis_title="Prix de clôture",
        template="plotly_dark"
    )
    fig.show()

In [35]:
# Liste des actions à comparer
actions = ["AAPL", "MSFT", "TSLA", "GOOGL", "AMZN", "NFLX", "META"]  # Liste prédéfinie
periode = "1y"  # Période d'historique
intervalle = "1d"  # Intervalle des données (1 jour)

# Fonction pour récupérer les données pour plusieurs actions
def fetch_multiple_stocks(actions, periode, intervalle):
    donnees = {}
    for symbole in actions:
        try:
            ticker = yf.Ticker(symbole)
            historique = ticker.history(period=periode, interval=intervalle)
            if not historique.empty:
                donnees[symbole] = historique["Close"]
            else:
                print(f"Aucune donnée trouvée pour {symbole}.")
        except Exception as e:
            print(f"Erreur pour {symbole} : {str(e)}")
    return pd.DataFrame(donnees)

# Récupération des données pour toutes les actions
df_comparaison = fetch_multiple_stocks(actions, periode, intervalle)
df_comparaison.head()  # Aperçu des données récupérées

,AAPL,MSFT,TSLA,GOOGL,AMZN,NFLX,META
Date,,,,,,,
2023-11-29 00:00:00-05:00,188.446808,376.038025,244.139999,134.656570,146.320007,477.190002,331.202820
2023-11-30 00:00:00-05:00,189.023972,376.097595,240.080002,132.202637,146.089996,473.970001,326.167938
2023-12-01 00:00:00-05:00,190.307678,371.730225,238.830002,131.534302,147.029999,465.739990,323.844971
2023-12-04 00:00:00-05:00,188.506485,366.400146,235.580002,128.950699,144.839996,453.899994,319.059357
2023-12-05 00:00:00-05:00,192.477051,369.755005,238.720001,130.666458,146.880005,455.149994,317.334564


In [36]:
# Normaliser les données (base 100)
df_normalise = df_comparaison / df_comparaison.iloc[0] * 100
df_normalise.head()

,AAPL,MSFT,TSLA,GOOGL,AMZN,NFLX,META
Date,,,,,,,
2023-11-29 00:00:00-05:00,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
2023-11-30 00:00:00-05:00,100.306274,100.015842,98.337021,98.177635,99.842803,99.325216,98.479819
2023-12-01 00:00:00-05:00,100.987478,98.854424,97.825019,97.681310,100.485232,97.600534,97.778446
2023-12-04 00:00:00-05:00,100.031668,97.436994,96.493816,95.762649,98.988511,95.119343,96.333527
2023-12-05 00:00:00-05:00,102.138663,98.329153,97.779963,97.036823,100.382721,95.381293,95.812760


In [37]:
# Calcul des variations en pourcentage
variations = df_comparaison.apply(
    lambda x: ((x.iloc[-1] - x.iloc[0]) / x.iloc[0]) * 100 if len(x) > 1 else None
)

# Afficher les variations
print("Variations en pourcentage sur la période :")
print(variations)

Variations en pourcentage sur la période :
AAPL     25.425314
MSFT     12.541813
TSLA     38.752356
GOOGL    25.424627
AMZN     40.932200
NFLX     86.961162
META     74.590905
dtype: float64


In [38]:
# Statistiques de prix et volume
prix_max = df_comparaison.max()
prix_min = df_comparaison.min()

# Affichage des statistiques
print("Prix maximums sur la période :")
print(prix_max)
print("Prix minimums sur la période :")
print(prix_min)

Prix maximums sur la période :
AAPL     236.360001
MSFT     465.786438
TSLA     352.559998
GOOGL    190.926636
AMZN     214.100006
NFLX     897.789978
META     595.940002
dtype: float64
Prix minimums sur la période :
AAPL     164.405121
MSFT     363.213928
TSLA     142.050003
GOOGL    128.950699
AMZN     144.520004
NFLX     446.730011
META     316.497070
dtype: float64


In [39]:
import plotly.graph_objects as go

fig = go.Figure()
for action in df_comparaison.columns:
    fig.add_trace(go.Scatter(
        x=df_comparaison.index,
        y=df_comparaison[action],
        mode='lines',
        name=action
    ))

fig.update_layout(
    title="Comparaison des Prix de Clôture des Actions",
    xaxis_title="Date",
    yaxis_title="Prix de Clôture",
    template="plotly_dark",
    hovermode="x unified"
)
fig.show()

In [40]:
fig_normalise = go.Figure()
for action in df_normalise.columns:
    fig_normalise.add_trace(go.Scatter(
        x=df_normalise.index,
        y=df_normalise[action],
        mode='lines',
        name=action
    ))

fig_normalise.update_layout(
    title="Comparaison des Tendances (Normalisées, Base 100)",
    xaxis_title="Date",
    yaxis_title="Prix Normalisé",
    template="plotly_dark",
    hovermode="x unified"
)
fig_normalise.show()

In [41]:
# Créer un tableau récapitulatif des performances
tableau_performance = pd.DataFrame({
    "Symbole": df_comparaison.columns,
    "Variation (%)": variations,
    "Prix Max": prix_max,
    "Prix Min": prix_min
}).reset_index(drop=True)

# Ajustement des formats numériques
tableau_performance["Variation (%)"] = tableau_performance["Variation (%)"].map(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A")
tableau_performance["Prix Max"] = tableau_performance["Prix Max"].map(lambda x: f"${x:.2f}" if pd.notnull(x) else "N/A")
tableau_performance["Prix Min"] = tableau_performance["Prix Min"].map(lambda x: f"${x:.2f}" if pd.notnull(x) else "N/A")

# Affichage du tableau
print("Tableau récapitulatif des performances :")
print(tableau_performance)

Tableau récapitulatif des performances :
  Symbole Variation (%) Prix Max Prix Min
0    AAPL        25.43%  $236.36  $164.41
1    MSFT        12.54%  $465.79  $363.21
2    TSLA        38.75%  $352.56  $142.05
3   GOOGL        25.42%  $190.93  $128.95
4    AMZN        40.93%  $214.10  $144.52
5    NFLX        86.96%  $897.79  $446.73
6    META        74.59%  $595.94  $316.50


## Indicateurs par pays ( API Banque Mondiale)

In [43]:
# Importation des bibliothèques nécessaires
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go

# Description du notebook
"""
Comment récupérer et analyser les données économiques clés par pays en utilisant l'API de la Banque mondiale.
"""

"\nComment récupérer et analyser les données économiques clés par pays en utilisant l'API de la Banque mondiale.\n"

In [44]:
# Paramètres utilisateur
indicateur = "NY.GDP.PCAP.CD"  # Exemple : PIB par habitant
annee = 2021  # Année sélectionnée

# Fonction pour récupérer les données
def fetch_indicator_data(indicateur, annee):
    url = f"https://api.worldbank.org/v2/country/all/indicator/{indicateur}?format=json&date={annee}&per_page=300"
    response = requests.get(url)
    data = response.json()
    if len(data) > 1:
        return pd.DataFrame(data[1])
    return pd.DataFrame()

# Appel de la fonction
df = fetch_indicator_data(indicateur, annee)
if df.empty:
    print("Aucune donnée disponible pour cet indicateur et cette année.")
else:
    df.head()  # Aperçu des données brutes

In [45]:
# Nettoyage des données
if not df.empty:
    df = df[['country', 'value']].dropna()
    df['Country'] = df['country'].apply(lambda x: x['value'])
    df['Country Code'] = df['country'].apply(lambda x: x['id'])
    df['Value'] = df['value'].astype(float)

    # Charger les données géographiques pour la carte
    df_geo = px.data.gapminder().query("year==2007")[['country', 'iso_alpha']].drop_duplicates()
    df = pd.merge(df, df_geo, left_on='Country', right_on='country', how='left')

    # Supprimer les pays sans code ISO
    df = df.dropna(subset=['iso_alpha'])
    df.head()  # Aperçu des données nettoyées

In [46]:
# Création de la carte
fig_map = px.choropleth(
    df,
    locations="iso_alpha",
    color="Value",
    hover_name="Country",
    color_continuous_scale=px.colors.sequential.Turbo,
    labels={'Value': "PIB par habitant (USD)"},
    projection='natural earth',
)

# Personnalisation de la carte
fig_map.update_geos(
    showcoastlines=False,
    showland=True,
    landcolor="#f0f0f0",
    showocean=True,
    oceancolor="#d1e7f7",
    showlakes=False,
)
fig_map.update_layout(
    title="Carte : PIB par habitant (USD) en 2021",
    margin={"r":0,"t":30,"l":0,"b":0},
)
fig_map.show()

In [47]:
# Top 10 des pays
df_top10 = df.sort_values(by='Value', ascending=False).head(10)

# Création de l'histogramme
fig_bar = go.Figure()
fig_bar.add_trace(go.Bar(
    x=df_top10['Value'][::-1],
    y=df_top10['Country'][::-1],
    orientation='h',
    marker=dict(
        color=df_top10['Value'][::-1],
        colorscale='Turbo',
    ),
    hovertemplate='<b>%{y}</b><br>Valeur: %{x:.2f}<extra></extra>',
))

# Personnalisation de l'histogramme
fig_bar.update_layout(
    title="Top 10 des pays par PIB par habitant (USD)",
    xaxis_title="PIB par habitant (USD)",
    yaxis_title="Pays",
    template='plotly_white',
    margin={"r":0,"t":30,"l":0,"b":0},
)
fig_bar.show()

In [48]:
# Pays sélectionnés
pays_selectionnes = ["United States", "France", "China", "India", "Germany"]

# Récupérer les codes pays
country_info = df[df['Country'].isin(pays_selectionnes)][['Country', 'Country Code']].drop_duplicates()
country_codes = country_info['Country Code'].tolist()

# Fonction pour récupérer les données historiques
def fetch_historical_data(indicateur, country_codes):
    frames = []
    for code in country_codes:
        url = f"https://api.worldbank.org/v2/country/{code}/indicator/{indicateur}?format=json&date=2000:2021&per_page=300"
        response = requests.get(url)
        data = response.json()
        if len(data) > 1:
            df_hist = pd.DataFrame(data[1])
            df_hist['Country Code'] = code
            frames.append(df_hist)
    if frames:
        return pd.concat(frames, ignore_index=True)
    return pd.DataFrame()

# Appel de la fonction
df_hist = fetch_historical_data(indicateur, country_codes)

# Nettoyage et visualisation
if not df_hist.empty:
    df_hist = df_hist[['country', 'date', 'value']].dropna()
    df_hist['Value'] = df_hist['value'].astype(float)
    df_hist['Year'] = df_hist['date'].astype(int)
    df_hist['Country'] = df_hist['country'].apply(lambda x: x['value'])

    # Création du graphique temporel
    fig_line = go.Figure()
    for country in pays_selectionnes:
        df_country = df_hist[df_hist['Country'] == country]
        fig_line.add_trace(go.Scatter(
            x=df_country['Year'],
            y=df_country['Value'],
            mode='lines+markers',
            name=country,
            line=dict(width=3),
        ))

    # Personnalisation du graphique
    fig_line.update_layout(
        title="Évolution du PIB par habitant (USD) au fil du temps",
        xaxis_title="Année",
        yaxis_title="PIB par habitant (USD)",
        template='plotly_white',
    )
    fig_line.show()
else:
    print("Aucune donnée historique disponible.")

In [ ]:
# Exporter les données nettoyées
#df_export = df[['Country', 'Value']]
#df_export.to_csv("indicateurs_pays.csv", index=False)
#print("Les données ont été exportées sous forme de fichier CSV : indicateurs_pays.csv")

## Cryptommonaies ( API Coin Geko)

In [49]:
# Importation des bibliothèques nécessaires
import pandas as pd
import requests
import plotly.graph_objects as go
from datetime import datetime, timedelta

# URL de l'API CoinGecko
COINGECKO_API = "https://api.coingecko.com/api/v3/coins/markets"

# Description
"""
Comment utiliser l'API CoinGecko pour analyser les principales cryptomonnaies :
- Récupération des données en temps réel.
- Visualisation avancée : prix historiques, heatmap, et graphique 3D.
"""

"\nComment utiliser l'API CoinGecko pour analyser les principales cryptomonnaies\u202f:\n- Récupération des données en temps réel.\n- Visualisation avancée\u202f: prix historiques, heatmap, et graphique 3D.\n"

In [60]:
# Fonction pour récupérer les données des cryptomonnaies
def recuperer_donnees_crypto(devise="usd", nb_cryptos=20):
    try:
        params = {
            "vs_currency": devise,
            "order": "market_cap_desc",
            "per_page": nb_cryptos,
            "page": 1,
            "sparkline": "true",
            "price_change_percentage": "1h,24h,7d"
        }
        response = requests.get(COINGECKO_API, params=params)
        if response.status_code == 200:
            return pd.DataFrame(response.json())
        else:
            print(f"Erreur {response.status_code} lors de la récupération des données.")
            return pd.DataFrame()
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
        return pd.DataFrame()

# Récupérer les données pour les 10 principales cryptomonnaies
devise = "usd"  # Monnaie utilisée (USD par défaut)
nb_cryptos = 10  # Nombre de cryptos à récupérer
data_crypto = recuperer_donnees_crypto(devise, nb_cryptos)

# Aperçu des données
data_crypto.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,sparkline_in_7d,price_change_percentage_1h_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_7d_in_currency
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,98193.000,1938715508924,1,2057355146756,66517926901,98685.000,...,2024-11-22T19:35:49.770Z,67.810000,144290.83887,2013-07-06T00:00:00.000Z,None,2024-11-29T16:25:05.135Z,"{'price': [98957.45944900438, 98685.7882146392...",-0.380604,3.129958,-0.247433
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,3606.340,433573895704,2,433573895704,28482420807,3649.170,...,2021-11-10T14:24:19.604Z,0.432979,831232.24982,2015-10-20T00:00:00.000Z,"{'times': 48.1207451740577, 'currency': 'btc',...",2024-11-29T16:25:09.959Z,"{'price': [3346.5455196347625, 3293.7397622031...",-0.907684,1.522389,9.159786
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,1.002,133428276706,3,133428276706,116000956237,1.003,...,2018-07-24T00:00:00.000Z,0.572521,74.46178,2015-03-02T00:00:00.000Z,None,2024-11-29T16:25:13.398Z,"{'price': [1.0008510517630425, 1.0009433245657...",-0.015373,0.180017,0.157059
3,solana,sol,Solana,https://coin-images.coingecko.com/coins/images...,243.460,115463539757,4,143244655969,5488585629,246.610,...,2024-11-23T15:05:59.896Z,0.500801,48417.21286,2020-05-11T19:35:23.449Z,None,2024-11-29T16:25:13.111Z,"{'price': [259.27140409899755, 259.39654086691...",-0.946752,3.714568,-4.623486
4,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,1.700,97070502033,5,170123217520,14031972100,1.760,...,2018-01-07T00:00:00.000Z,0.002686,63285.85689,2014-05-22T00:00:00.000Z,None,2024-11-29T16:25:15.237Z,"{'price': [1.4637653602118972, 1.4597012674227...",-2.873193,17.307849,17.961130


In [61]:
# Nettoyage des données
if not data_crypto.empty:
    data_crypto = data_crypto[[
        "name", "symbol", "current_price", "price_change_percentage_1h_in_currency",
        "price_change_percentage_24h_in_currency", "price_change_percentage_7d_in_currency",
        "market_cap", "total_volume", "sparkline_in_7d"
    ]]
    data_crypto.columns = [
        "Nom", "Symbole", "Prix Actuel", "Variation 1h (%)",
        "Variation 24h (%)", "Variation 7j (%)", "Capitalisation", "Volume Échangé", "Sparkline"
    ]
    data_crypto.head()  # Aperçu des données nettoyées

In [62]:
data_crypto.head()# Aperçu des données nettoyées

,Nom,Symbole,Prix Actuel,Variation 1h (%),Variation 24h (%),Variation 7j (%),Capitalisation,Volume Échangé,Sparkline
0,Bitcoin,btc,98193.000,-0.380604,3.129958,-0.247433,1938715508924,66517926901,"{'price': [98957.45944900438, 98685.7882146392..."
1,Ethereum,eth,3606.340,-0.907684,1.522389,9.159786,433573895704,28482420807,"{'price': [3346.5455196347625, 3293.7397622031..."
2,Tether,usdt,1.002,-0.015373,0.180017,0.157059,133428276706,116000956237,"{'price': [1.0008510517630425, 1.0009433245657..."
3,Solana,sol,243.460,-0.946752,3.714568,-4.623486,115463539757,5488585629,"{'price': [259.27140409899755, 259.39654086691..."
4,XRP,xrp,1.700,-2.873193,17.307849,17.961130,97070502033,14031972100,"{'price': [1.4637653602118972, 1.4597012674227..."


In [69]:
# Inspecter les données sparkline
print("Données Sparkline brutes pour chaque crypto :")
for _, row in data_crypto.iterrows():
    print(f"{row['Nom']} : {row.get('Sparkline', 'Sparkline non disponible')}")

Données Sparkline brutes pour chaque crypto :
Bitcoin : {'price': [98957.45944900438, 98685.78821463924, 97871.01435703253, 97393.85345172636, 98436.20344002808, 99036.38730170106, 99137.77149298869, 99366.40761088874, 99531.18012036204, 99334.42990748059, 99418.67011453414, 98853.46617956243, 98907.79325258995, 98721.12064236584, 98545.74682604708, 98610.3477536692, 98720.12576397, 98479.71802325314, 98803.60701519938, 98827.5462630559, 98688.35022303325, 98562.42427541426, 98567.61071655307, 98627.69963826089, 98573.42641332174, 98816.16419710916, 98638.74802471796, 98907.83827505098, 98637.06301800392, 97887.14429180366, 97993.79314280888, 97959.65085898795, 97627.80481379767, 97828.71753790964, 98078.20284165346, 97795.77021667767, 97903.58813942045, 98040.73700095096, 98292.35524999183, 98497.71417618614, 98293.90548790821, 98300.80802932262, 98321.94620028995, 98473.08914761616, 98553.76752974809, 98207.04580972278, 98047.89454351828, 97865.55396535275, 97446.22464625575, 96915.2

In [93]:
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
import requests

# URL de l'API CoinGecko
COINGECKO_API = "https://api.coingecko.com/api/v3/coins/markets"

# Fonction pour récupérer les données depuis l'API
def recuperer_donnees_crypto(devise="usd", nb_cryptos=10):
    params = {
        "vs_currency": devise,
        "order": "market_cap_desc",
        "per_page": nb_cryptos,
        "page": 1,
        "sparkline": "true",
        "price_change_percentage": "1h,24h,7d"
    }
    response = requests.get(COINGECKO_API, params=params)
    if response.status_code == 200:
        data = pd.DataFrame(response.json())
        return data
    else:
        print(f"Erreur {response.status_code} lors de la récupération des données.")
        return pd.DataFrame()

# Récupérer les données depuis l'API
df = recuperer_donnees_crypto(devise="usd", nb_cryptos=10)

# Vérification des données
if df.empty:
    print("Erreur : Aucun résultat n'a été récupéré.")
else:
    print("Données récupérées avec succès.")
    # Afficher les colonnes disponibles pour débuguer
    print("Colonnes disponibles :", df.columns)

    # Vérifier si la colonne sparkline_in_7d est présente
    if "sparkline_in_7d" not in df.columns:
        print("Erreur : La colonne 'sparkline_in_7d' est absente.")
    else:
        # Initialisation du graphique
        fig = go.Figure()

        # Ajout des données Sparkline au graphique
        for _, row in df.iterrows():
            sparkline_data = row["sparkline_in_7d"]
            if not sparkline_data or not isinstance(sparkline_data, dict) or "price" not in sparkline_data:
                print(f"Sparkline vide ou non valide pour {row['name']}.")
                continue

            sparkline = sparkline_data["price"]  # Liste des prix
            timestamps = [datetime.now() - timedelta(minutes=5 * (len(sparkline) - i)) for i in range(len(sparkline))]

            # Normaliser les données Sparkline (facultatif)
            sparkline_normalisee = [(price / sparkline[0]) * 100 for price in sparkline]

            # Ajout de la courbe au graphique
            fig.add_trace(go.Scatter(
                x=timestamps,
                y=sparkline_normalisee,
                mode="lines",
                name=row["name"],
                line=dict(width=2)
            ))

        # Mise en page du graphique
        fig.update_layout(
            title="Prix Historiques Normalisés sur 7 Jours",
            xaxis_title="Temps",
            yaxis_title="Prix Normalisé (Base 100)",
            legend_title="Cryptos",
            template="plotly_dark",
            xaxis=dict(rangeslider=dict(visible=True), type="date"),
            hovermode="x unified"
        )

        # Afficher le graphique
        fig.show()

Données récupérées avec succès.
Colonnes disponibles : Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated', 'sparkline_in_7d',
       'price_change_percentage_1h_in_currency',
       'price_change_percentage_24h_in_currency',
       'price_change_percentage_7d_in_currency'],
      dtype='object')


In [64]:
# Heatmap des variations
variations = data_crypto[[
    "Nom", "Variation 1h (%)", "Variation 24h (%)", "Variation 7j (%)"
]].set_index("Nom")

fig2 = go.Figure(data=go.Heatmap(
    z=variations.values,
    x=variations.columns,
    y=variations.index,
    colorscale="RdYlGn",
    reversescale=True,
    hoverongaps=False
))
fig2.update_layout(
    title="Heatmap des Variations (%)",
    xaxis_title="Période",
    yaxis_title="Cryptomonnaie",
    template="plotly_dark"
)
fig2.show()

In [66]:
# Graphique 3D des cryptos
fig3 = go.Figure()
fig3.add_trace(go.Scatter3d(
    x=data_crypto["Capitalisation"],
    y=data_crypto["Volume Échangé"],
    z=data_crypto["Prix Actuel"],
    mode='markers+text',
    text=data_crypto["Symbole"].str.upper(),
    marker=dict(
        size=8,
        color=data_crypto["Prix Actuel"],
        colorscale='Viridis',
        opacity=0.8,
        colorbar=dict(title='Prix Actuel')
    )
))
fig3.update_layout(
    scene=dict(
        xaxis_title="Capitalisation",
        yaxis_title="Volume Échangé",
        zaxis_title="Prix Actuel",
        bgcolor='#0e1117'
    ),
    title="Graphique 3D des Cryptomonnaies",
    template="plotly_dark"
)
fig3.show()

In [67]:
# Tableau récapitulatif
data_crypto_table = data_crypto[[
    "Nom", "Symbole", "Prix Actuel", "Variation 1h (%)",
    "Variation 24h (%)", "Variation 7j (%)", "Capitalisation", "Volume Échangé"
]]
data_crypto_table.style.format({
    "Prix Actuel": "${:.2f}",
    "Variation 1h (%)": "{:.2f}%",
    "Variation 24h (%)": "{:.2f}%",
    "Variation 7j (%)": "{:.2f}%",
    "Capitalisation": "${:,.0f}",
    "Volume Échangé": "${:,.0f}"
})

,Nom,Symbole,Prix Actuel,Variation 1h (%),Variation 24h (%),Variation 7j (%),Capitalisation,Volume Échangé
0,Bitcoin,btc,$98193.00,-0.38%,3.13%,-0.25%,"$1,938,715,508,924","$66,517,926,901"
1,Ethereum,eth,$3606.34,-0.91%,1.52%,9.16%,"$433,573,895,704","$28,482,420,807"
2,Tether,usdt,$1.00,-0.02%,0.18%,0.16%,"$133,428,276,706","$116,000,956,237"
3,Solana,sol,$243.46,-0.95%,3.71%,-4.62%,"$115,463,539,757","$5,488,585,629"
4,XRP,xrp,$1.70,-2.87%,17.31%,17.96%,"$97,070,502,033","$14,031,972,100"
5,BNB,bnb,$658.65,-0.36%,1.69%,5.46%,"$95,937,301,825","$1,538,506,720"
6,Dogecoin,doge,$0.42,-0.75%,5.70%,4.45%,"$61,329,334,856","$5,702,580,290"
7,USDC,usdc,$1.00,-0.01%,0.11%,0.27%,"$39,407,798,320","$5,719,904,142"
8,Cardano,ada,$1.06,-2.55%,7.67%,13.77%,"$37,934,469,304","$2,390,608,645"
9,Lido Staked Ether,steth,$3602.46,-0.79%,1.49%,9.15%,"$35,269,271,840","$72,478,865"


In [ ]:
# Exporter les données en CSV
#data_crypto.to_csv("cryptomonnaies.csv", index=False)
#print("Les données ont été exportées sous forme de fichier CSV : cryptomonnaies.csv")

## Actualités financières

In [94]:
import requests
import pandas as pd

In [95]:
# Clé API et URL de base
NEWS_API_KEY = "pub_597651f94f090c7a836fda48dacc8b8b8950b"
BASE_URL = "https://newsdata.io/api/1/news"

# Catégories valides
CATEGORIES_VALIDES = {
    "Entreprises": "business",
    "Technologie": "technology",
    "Science": "science",
    "Santé": "health",
    "Monde": "world"
}

In [96]:
def recuperer_actualites(categorie, langue="en"):
    """
    Récupère les actualités pour une catégorie et une langue données.
    """
    try:
        params = {
            "apikey": NEWS_API_KEY,
            "category": categorie,
            "language": langue
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            return data.get("results", [])
        else:
            print(f"Erreur {response.status_code}: {response.json().get('message', 'Impossible de récupérer les actualités.')}")
            return []
    except Exception as e:
        print(f"Erreur lors de l'appel à l'API : {e}")
        return []

In [97]:
# Exemple : Récupérer les actualités de la catégorie "business" en anglais
categorie = "business"
langue = "en"
actualites = recuperer_actualites(categorie, langue)

# Vérifier les résultats
if actualites:
    print(f"Nombre d'articles récupérés : {len(actualites)}")
else:
    print("Aucune actualité trouvée.")

Nombre d'articles récupérés : 10


In [98]:
# Transformer les données en DataFrame
if actualites:
    df = pd.DataFrame(actualites)
    # Afficher les colonnes disponibles
    print("Colonnes disponibles :", df.columns.tolist())
    # Afficher un aperçu
    display(df.head())
else:
    print("Pas de données à afficher.")

Colonnes disponibles : ['article_id', 'title', 'link', 'keywords', 'creator', 'video_url', 'description', 'content', 'pubDate', 'pubDateTZ', 'image_url', 'source_id', 'source_priority', 'source_name', 'source_url', 'source_icon', 'language', 'country', 'category', 'ai_tag', 'sentiment', 'sentiment_stats', 'ai_region', 'ai_org', 'duplicate']


,article_id,title,link,keywords,creator,video_url,description,content,pubDate,pubDateTZ,...,source_icon,language,country,category,ai_tag,sentiment,sentiment_stats,ai_region,ai_org,duplicate
0,d2f171514b4a6f12b059d66eacaa13ee,Activewear Market Size Worth USD 771.8 billion...,https://www.openpr.com/news/3763038/activewear...,"[advertising, media consulting, marketing rese...",[Allied Market Research],None,The activewear market size was valued at $425....,ONLY AVAILABLE IN PAID PLANS,2024-11-29 05:32:00,UTC,...,None,english,[germany],[business],ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN CORPORATE PLANS,ONLY AVAILABLE IN CORPORATE PLANS,False
1,214896af1b5511b1d13e4606ca71f5c9,Morning Bid: Yen bulls charge ahead after Toky...,https://finance.yahoo.com/news/morning-bid-yen...,None,[Reuters],None,None,ONLY AVAILABLE IN PAID PLANS,2024-11-29 05:31:57,UTC,...,https://i.bytvi.com/domain_icons/yahoo_sg.png,english,[singapore],[business],ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN CORPORATE PLANS,ONLY AVAILABLE IN CORPORATE PLANS,False
2,589b0052f404da2620364adf6195d734,"South Africa: Over 1,000 Spaza Shops Shut Down...",https://allafrica.com/stories/202411290059.html,"[south africa, women and gender, governance, t...",[info@allafrica.com (allAfrica)],None,[allAfrica],ONLY AVAILABLE IN PAID PLANS,2024-11-29 05:31:57,UTC,...,https://i.bytvi.com/domain_icons/allafrica.png,english,[tanzania],[business],ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN CORPORATE PLANS,ONLY AVAILABLE IN CORPORATE PLANS,False
3,22e3b58c979425048006319ec7903771,Morning Bid: Yen bulls charge ahead after Toky...,https://ca.finance.yahoo.com/news/morning-bid-...,None,[Reuters],None,None,ONLY AVAILABLE IN PAID PLANS,2024-11-29 05:31:57,UTC,...,https://i.bytvi.com/domain_icons/yahoo.png,english,[canada],[business],ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN CORPORATE PLANS,ONLY AVAILABLE IN CORPORATE PLANS,True
4,4908c70801bd01eee573d7ab2ee7e228,Samsung Galaxy S25 Ultra: Top Features and Price,https://www.analyticsinsight.net/photo/samsung...,"[ 200mp camera , snapdragon 8 elite, 120hz d...",None,None,None,ONLY AVAILABLE IN PAID PLANS,2024-11-29 05:31:34,UTC,...,https://i.bytvi.com/domain_icons/analyticsinsi...,english,[united states of america],"[technology, business]",ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN PROFESSIONAL AND CORPORATE P...,ONLY AVAILABLE IN CORPORATE PLANS,ONLY AVAILABLE IN CORPORATE PLANS,False


In [99]:
# Afficher uniquement les colonnes essentielles
if actualites:
    colonnes_essentielles = ["title", "description", "link", "pubDate"]
    if all(col in df.columns for col in colonnes_essentielles):
        df_essentiel = df[colonnes_essentielles]
        display(df_essentiel)
    else:
        print("Colonnes essentielles manquantes.")

,title,description,link,pubDate
0,Activewear Market Size Worth USD 771.8 billion...,The activewear market size was valued at $425....,https://www.openpr.com/news/3763038/activewear...,2024-11-29 05:32:00
1,Morning Bid: Yen bulls charge ahead after Toky...,None,https://finance.yahoo.com/news/morning-bid-yen...,2024-11-29 05:31:57
2,"South Africa: Over 1,000 Spaza Shops Shut Down...",[allAfrica],https://allafrica.com/stories/202411290059.html,2024-11-29 05:31:57
3,Morning Bid: Yen bulls charge ahead after Toky...,None,https://ca.finance.yahoo.com/news/morning-bid-...,2024-11-29 05:31:57
4,Samsung Galaxy S25 Ultra: Top Features and Price,None,https://www.analyticsinsight.net/photo/samsung...,2024-11-29 05:31:34
5,Horse Betting Market In 2024 : Competitive Ana...,(MENAFN - EIN Presswire) The horse betting mar...,https://menafn.com/1108938942/Horse-Betting-Ma...,2024-11-29 05:31:32
6,"Yobe support 15,000 small scale business owner...","Yobe State government has supported 15,000 sma...",https://dailytrust.com/yobe-support-15000-smal...,2024-11-29 05:31:03
7,New Gen Honda Amaze Variants & Top-Spec Featur...,New Gen Honda Amaze Variants & Top-Spec Featur...,https://news.google.com/rss/articles/CBMihgFBV...,2024-11-29 05:30:38
8,A' Landscape Planning and Garden Design Award ...,The A' Landscape Planning and Garden Design Aw...,https://www.openpr.com/news/3763036/a-landscap...,2024-11-29 05:30:36
9,Accounting and Budgeting Software Market 2033:...,The Business Research Company recently release...,https://www.openpr.com/news/3763035/accounting...,2024-11-29 05:30:23


In [ ]:
# Sauvegarder les données dans un fichier CSV
#if actualites:
    #df_essentiel.to_csv("actualites.csv", index=False)
    #print("Fichier 'actualites.csv' sauvegardé avec succès.")